In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install transformers
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


#Importing Libraries, Data and Summarizer


In [3]:

import pandas as pd
import numpy as np
from bert_score import score
import json
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline


#Importing the Data


In [4]:
# Open the JSON file
with open('/content/drive/MyDrive/scraped_articles.json', 'r') as f:
    # Load the JSON data
    data = json.load(f)

# Convert the JSON data to a DataFrame
df = pd.json_normalize(data)
df.head

<bound method NDFrame.head of                                                  title            author  \
0    Health News - latest stories and features for ...                     
1    I’m a 63-year-old eight-time bodybuilding cham...       Iona Cleave   
2    I’m a 63-year-old eight-time bodybuilding cham...       Iona Cleave   
3    I’m a 63-year-old eight-time bodybuilding cham...       Iona Cleave   
4    I’m a 63-year-old eight-time bodybuilding cham...       Iona Cleave   
..                                                 ...               ...   
181  Untangling Rosalind Franklin’s Role in DNA Dis...      Emily Anthes   
182  Here Are the Places Most at Risk From Record-S...     Raymond Zhong   
183                          Tucker Carlson’s Downfall      German Lopez   
184  A grain glut leaves some Eastern European coun...    Andrew Higgins   
185  The Group That Searches for Missing Ukrainian ...  Julian E. Barnes   

                    date                                 

#DATA CLEANING 

In [5]:
# Identify the column(s) that contain the article content
content_columns = ['title', 'content']

# Identify duplicate rows based on the article content
duplicates = df.duplicated(subset=content_columns, keep='first')

# Remove the duplicate rows from the DataFrame
df = df[~duplicates]

# Print the number of rows remaining in the DataFrame
print(len(df))
df.head


96


<bound method NDFrame.head of                                                  title  \
0    Health News - latest stories and features for ...   
1    I’m a 63-year-old eight-time bodybuilding cham...   
5    Inside the conflict engulfing Sudan – a nation...   
9    Elite UK troops arrive in Sudan to aid evacuat...   
13   Moment ‘test dummy’ YouTuber nearly blows his ...   
..                                                 ...   
181  Untangling Rosalind Franklin’s Role in DNA Dis...   
182  Here Are the Places Most at Risk From Record-S...   
183                          Tucker Carlson’s Downfall   
184  A grain glut leaves some Eastern European coun...   
185  The Group That Searches for Missing Ukrainian ...   

                          author                 date  \
0                                                None   
1                    Iona Cleave  2023-04-25 23:28:32   
5    Nick Parker, Jerome Starkey  2023-04-25 22:32:30   
9    Nick Parker, Jerome Starkey  2023-04-25 

In [6]:
# Remove articles with less than or equal to 30 words
df = df[df['content'].str.split().str.len() > 30]
# Print the number of rows remaining in the DataFrame
print(len(df))
# reset the index of the dataset to start at 0
df = df.reset_index(drop=True)

df.head()

95


,title,author,date,content,source
0,I’m a 63-year-old eight-time bodybuilding cham...,Iona Cleave,2023-04-25 23:28:32,A BODYBUILDING legend has made a heartfelt rev...,https://www.thesun.co.uk
1,Inside the conflict engulfing Sudan – a nation...,"Nick Parker, Jerome Starkey",2023-04-25 22:32:30,GOLD-rich Sudan is the coup-capital of Africa....,https://www.thesun.co.uk
2,Elite UK troops arrive in Sudan to aid evacuat...,"Nick Parker, Jerome Starkey",2023-04-25 22:23:38,CRACK Commandos arrived in Sudan yesterday to ...,https://www.thesun.co.uk
3,Moment ‘test dummy’ YouTuber nearly blows his ...,Iona Cleave,2023-04-25 20:42:05,A DANGEROUS stunt YouTuber almost blew his hea...,https://www.thesun.co.uk
4,I was a hostage on Pan Am Flight 73 – I never ...,Grant Rollings,2023-04-25 20:25:05,KNEELING at the front of a hijacked jumbo jet ...,https://www.thesun.co.uk


#DOWNLOAD BART

In [7]:
# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to("cuda")  # Move the model to GPU

# Create the summarization pipeline with the GPU-enabled model
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)  # Set device to 0 for GPU

#Testing the model on the first article


In [8]:
test = df['content'][0]
pipe_out = pipe(test, max_length=120, min_length=100, do_sample=False)



In [9]:
pipe_out

[{'summary_text': "Lee Haney, 63, won eight Mr Olympia titles in a row between 1984-1991. The bodybuilder even beat Arnold Schwarzenegger's record of seven wins. He took to Instagram to reveal that being a father is his greatest reward. He posted an adorable painting of him holding his daughter as a small baby. Haney retired at 31, before turning to personal training and working with athletes like boxer Evander Holyfield. He now posts a lot about his family online as well as offering tips and advice to his keen-to-learn followers."}]

In [10]:
summarized_text = pipe_out[0]['summary_text']

#check the word length of summarized text
len(summarized_text.split())


88

# SUMMARIZING ARTICLES

In [12]:
max_seq_length = 1024  # set the maximum sequence length for the model
segment_length = 500  # set the desired length for each text segment

# loop over the articles and summarize each one
summarized_articles = []
summary_cache = {}  # Initialize the caching mechanism

for article in df['content']:
    # Check if the article is already summarized in the cache
    if article in summary_cache:
        summarized_article = summary_cache[article]
    else:
        # split the article into smaller segments
        segments = [article[i:i+segment_length] for i in range(0, len(article), segment_length)]
        
        # summarize each segment separately and concatenate the summaries
        segment_summaries = []
        for segment in segments:
            if len(segment.split()) < 100:
                pipe_out = pipe(segment, max_length=40, min_length=20, do_sample=False)
                segment_summaries.append(pipe_out[0]['summary_text'])
            
            else:
                pipe_out = pipe(segment, max_length=120, min_length=100, do_sample=False)
                segment_summaries.append(pipe_out[0]['summary_text'])
    
        summarized_article = ' '.join(segment_summaries)
        summary_cache[article] = summarized_article  # Store the summary in the cache

    summarized_articles.append(summarized_article)

Your max_length is set to 40, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 40, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 40, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 40, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 40, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 40, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 40, but you input_length is only 31. You might consider decreasing

In [13]:
df['summarized_articles'] = summarized_articles
df.tail()

,title,author,date,content,source,summarized_articles
90,Untangling Rosalind Franklin’s Role in DNA Dis...,Emily Anthes,2023-04-25 00:00:00,This account became a parable of poor scientif...,https://www.nytimes.com,Dr. Franklin's account became a parable of poo...
91,Here Are the Places Most at Risk From Record-S...,Raymond Zhong,2023-04-25 00:00:00,Dr. Mitchell and his colleagues looked at maxi...,https://www.nytimes.com,31 percent of Earth’s land surface experienced...
92,Tucker Carlson’s Downfall,German Lopez,2023-04-25 00:00:00,Carlson did so by embracing Trumpism but not T...,https://www.nytimes.com,Carlson embraced Trumpism but not Trump himsel...
93,A grain glut leaves some Eastern European coun...,Andrew Higgins,2023-04-25 00:00:00,After more than a year of surprisingly solid E...,https://www.nytimes.com,A glut of Ukrainian cereals and other produce ...
94,The Group That Searches for Missing Ukrainian ...,Julian E. Barnes,2023-04-25 00:00:00,WASHINGTON — When the Russian Army took Kherso...,https://www.nytimes.com,Anastasia Kuznetsova was offered a chance to g...


Clean it again

In [14]:
# Define a function to clean text
def clean_text(text):
    # Remove \n
    text = re.sub('\n', ' ', text)
    # Remove \u2019 and \u2026
    text = re.sub('\u2019|\u2026', '', text)
    # Remove any other unwanted characters or symbols
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    return text

# Clean the text data in the JSON file
data['content'] = clean_text(data['content'])
data['summarized_articles'] = clean_text(data['summarized_articles'])
data['title'] = clean_text(data['title'])


TypeError: ignored

In [17]:
df['summarized_length'] = df['summarized_articles'].apply(lambda x: len(x.split()))
df['article_length'] = df['content'].apply(lambda x: len(x.split()))
df.tail()

,title,author,date,content,source,summarized_articles,summarized_length,article_length
90,Untangling Rosalind Franklin’s Role in DNA Dis...,Emily Anthes,2023-04-25 00:00:00,This account became a parable of poor scientif...,https://www.nytimes.com,Dr. Franklin's account became a parable of poo...,107,283
91,Here Are the Places Most at Risk From Record-S...,Raymond Zhong,2023-04-25 00:00:00,Dr. Mitchell and his colleagues looked at maxi...,https://www.nytimes.com,31 percent of Earth’s land surface experienced...,99,242
92,Tucker Carlson’s Downfall,German Lopez,2023-04-25 00:00:00,Carlson did so by embracing Trumpism but not T...,https://www.nytimes.com,Carlson embraced Trumpism but not Trump himsel...,101,289
93,A grain glut leaves some Eastern European coun...,Andrew Higgins,2023-04-25 00:00:00,After more than a year of surprisingly solid E...,https://www.nytimes.com,A glut of Ukrainian cereals and other produce ...,38,173
94,The Group That Searches for Missing Ukrainian ...,Julian E. Barnes,2023-04-25 00:00:00,WASHINGTON — When the Russian Army took Kherso...,https://www.nytimes.com,Anastasia Kuznetsova was offered a chance to g...,49,173


In [18]:
# save the dataframe to a JSON file
df.to_json('/content/drive/MyDrive/summarized_article_interntest.json')

CALCULATING THE BERT SCORE

In [19]:
#calcualte bert score
P, R, F1 = score(list(df['content']), list(df['summarized_articles']), lang='en', verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 13.64 seconds, 6.96 sentences/sec


In [20]:
df['bert_f1_score'] = F1
df['bert_P_score'] = P
df['bert_R_score'] = R


In [21]:
df.head()


,title,author,date,content,source,summarized_articles,summarized_length,article_length,bert_f1_score,bert_P_score,bert_R_score
0,I’m a 63-year-old eight-time bodybuilding cham...,Iona Cleave,2023-04-25 23:28:32,A BODYBUILDING legend has made a heartfelt rev...,https://www.thesun.co.uk,"Lee Haney, 63, took to Instagram to tell his l...",191,579,0.868734,0.842292,0.896889
1,Inside the conflict engulfing Sudan – a nation...,"Nick Parker, Jerome Starkey",2023-04-25 22:32:30,GOLD-rich Sudan is the coup-capital of Africa....,https://www.thesun.co.uk,Once home to ancient kingdoms of pyramid-build...,133,387,0.864891,0.824172,0.909843
2,Elite UK troops arrive in Sudan to aid evacuat...,"Nick Parker, Jerome Starkey",2023-04-25 22:23:38,CRACK Commandos arrived in Sudan yesterday to ...,https://www.thesun.co.uk,The elite UK troops flew in from Cyprus to pro...,285,905,0.863695,0.853505,0.874131
3,Moment ‘test dummy’ YouTuber nearly blows his ...,Iona Cleave,2023-04-25 20:42:05,A DANGEROUS stunt YouTuber almost blew his hea...,https://www.thesun.co.uk,Shards from the blast hit Zo Choppa's temple a...,126,348,0.870344,0.831524,0.912965
4,I was a hostage on Pan Am Flight 73 – I never ...,Grant Rollings,2023-04-25 20:25:05,KNEELING at the front of a hijacked jumbo jet ...,https://www.thesun.co.uk,"Zaid Hassan Abd Latif Safarini, of the feared ...",343,1039,0.845572,0.840800,0.850398
